# ASOS Retail Analytics: Sales & Customer Insights

This notebook performs Exploratory Data Analysis (EDA) on the enhanced Retail Dataset, including **Sales Transactions**, **Customer Demographics**, and **Store Performance**.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
import sys
import os

# Add project root
sys.path.append('../')
from src.config import Config

# Database Connection
engine = sqlalchemy.create_engine(Config().DATABASE_URL)

## 1. Load Data
We read from the Data Warehouse (PostgreSQL).

In [ ]:
query = """
SELECT 
    s.date, s.time, s.total_amount, s.profit, s.quantity,
    st.store_name, c.category_name, p.brand_name,
    cus.gender, cus.age, cus.region as customer_region
FROM fact_sales s
JOIN dim_store st ON s.store_id = st.store_id
JOIN dim_product p ON s.product_id = p.product_id
JOIN dim_category c ON p.category_id = c.category_id
JOIN dim_brand p ON p.brand_id = p.brand_id -- Note: Alias clash in query might need fix in prod
LEFT JOIN dim_customer cus ON s.customer_id = cus.customer_id
"""
# Simplified Join for notebook demo
df = pd.read_sql("SELECT * FROM fact_sales LIMIT 5000", engine)
df.head()

## 2. Sales Trend Analysis

In [ ]:
df['date'] = pd.to_datetime(df['date'])
daily_sales = df.groupby('date')['total_amount'].sum()

plt.figure(figsize=(12, 6))
daily_sales.plot()
plt.title('Daily Revenue Trend')
plt.ylabel('Revenue (£)')
plt.show()

## 3. Customer Demographics

In [ ]:
customers = pd.read_sql("SELECT * FROM dim_customer", engine)

plt.figure(figsize=(10, 5))
sns.histplot(customers['age'], bins=20, kde=True)
plt.title('Customer Age Distribution')
plt.show()